[Reference](https://medium.com/geekculture/create-an-interactive-report-by-using-plotly-981a513fdec4)

In [2]:
import pandas as pd
import numpy as np
import random
import datetime
import re
import string
# set the seed
random.seed(1994)
# Generate Website Log Data
df = pd.DataFrame()
# choose the data length
rows = 2000

website_li = ['Amazon','taobao']
random_index = np.random.randint(0,len(website_li),size=rows).tolist()
website = [website_li[i] for i in random_index]

pageVersion_li = ['web','mobile', 'app']
random_index = np.random.randint(0,len(pageVersion_li),size=rows).tolist()
pageVersion = [pageVersion_li[i] for i in random_index]

actionID_li = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
random_index = np.random.randint(0,len(actionID_li),size=rows).tolist()
actionID = [actionID_li[i] for i in random_index]

actionID_li = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

def date_generater(start_time, end_time):
    # confirm input is string type
    if not(isinstance(start_time, str) and isinstance(end_time, str)):
        raise TypeError('date format must be a string')
        
    # confirm date format is correct  
    if (re.fullmatch(pattern=r'\d{4}(0[1-9]|1[0-2])([0-2][0-9]|3[0-1])' , string = start_time, flags=0) == None) or (
        re.fullmatch(pattern=r'\d{4}(0[1-9]|1[0-2])([0-2][0-9]|3[0-1])' , string = end_time, flags=0) == None):
        raise ValueError('date format is wrong')

    year, month, day = int(start_time[0:4]), int(start_time[4:6]), int(start_time[6:8])
    start = datetime.date(year, month, day)

    year, month, day = int(end_time[0:4]), int(end_time[4:6]), int(end_time[6:8])
    end = datetime.date(year, month, day)

    date_list = [start + datetime.timedelta(days=i) for i in range((end - start).days+1)]
    return date_list
    
datetime_li = date_generater('20200101', '20200110')
random_index = np.random.randint(0,len(datetime_li),size=rows).tolist()
datetime = [datetime_li[i] for i in random_index]

df['datetime'] = datetime
df['website'] = website
df['page_version'] = pageVersion
df['actionID'] = actionID

df.to_pickle("./web_lod_data.pkl")

# Generate Search Items on Internal Website
letter_li = list(string.ascii_letters)*10
weights = np.random.randint(0,10,size=len(letter_li)).tolist()
random_letter_li = random.choices(letter_li, weights=weights, k=len(letter_li))

word_li = ['Test']*len(letter_li)
word_li = [word+letter for word, letter in zip(word_li, random_letter_li)]
df = pd.DataFrame(word_li, columns=['keyword'])
df.to_pickle("./keyword.pkl")

In [5]:
# plotly
import plotly.graph_objects as go
import plotly
from plotly.subplots import make_subplots
# basic package
import pandas as pd
import glob
# import config
import numpy as np
import re
from datetime import date
import os
import pickle
# wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
# plot
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import base64
from io import BytesIO
from os import path
from PIL import Image
# from pivottablejs import pivot_ui
from IPython.display import IFrame
import json, io
# html
from IPython.display import HTML
df = pd.read_pickle('./web_lod_data.pkl')
df.head()

,datetime,website,page_version,actionID
0,2020-01-03,Amazon,app,8
1,2020-01-10,Amazon,app,2
2,2020-01-05,taobao,mobile,7
3,2020-01-02,taobao,app,3
4,2020-01-10,taobao,app,5


In [6]:
df_group = df.groupby(['datetime', 'website', 'page_version']).size().reset_index(name='size')
df_group = df_group.sort_values(by='datetime')
idsite_pageversion_group = df_group.groupby(['website', 'page_version'])
idsite_pageversion_group_key = idsite_pageversion_group.groups.keys()

all_group_datetime_li = []
all_group_action_count_li = []
for key in idsite_pageversion_group_key:
    group = idsite_pageversion_group.get_group(key)
    datetime_li = sorted(list(set(group['datetime'])))
    action_count_li = list(group['size'])
    all_group_datetime_li.append(datetime_li)
    all_group_action_count_li.append(action_count_li)
    
group_key_li = list(idsite_pageversion_group_key)
idsite_li = [i[0] for i in group_key_li]
pageVersion_li = [i[1] for i in group_key_li]


li=[]
for idsite in ['Amazon', 'taobao']:
    li.append(dict(label=str(idsite),
                method="update",
                args=[{"visible": [i== idsite for i in idsite_li]},
                      {"title": f"Action Count on {idsite} website",
                       "annotations": [],
                       "title_x": 0.5
                      },
                     ]
               ))
  
  
# Initialize figure
fig = go.Figure()
for datetime, value, idsite, pageVersion in zip(all_group_datetime_li, all_group_action_count_li, idsite_li, pageVersion_li):
    if idsite == 'Amazon':
        fig.add_trace(
            go.Scatter(x=datetime,
                       y=value,
                       name=pageVersion,
#                        line=dict(color="#FFA500"),
                       visible=True
                      ))
    elif idsite == 'taobao':
        fig.add_trace(
            go.Scatter(x=datetime,
                       y=value,
                       name=pageVersion,
#                        line=dict(color="#33CFA5"),
                       visible=False
                      ))

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list(li),
            bordercolor = "#BEC8D9",  # default
            type="buttons",  # "dropdown" 
            direction="down",
            showactive=True, # Highlights active dropdown item or active button if True.    
        )
    ])   
        
# Set title
fig.update_layout(
                 title={
        'text': f"Action Count on {li[0]['label']} website",
        'font':{'size':30}
                 })
# fig.update_layout(title_font_color="#FFA500")
fig.update_layout(title_x=0.5)
plotly.offline.plot(fig, auto_open=False, filename=f"./action_count.html")

'./action_count.html'

In [8]:
keyword_search_in_website = pd.read_pickle('./keyword.pkl')
keyword_search_in_website = ' '.join(keyword_search_in_website['keyword'].tolist())
keyword_li = keyword_search_in_website.split()

from collections import Counter
keyword_count = Counter(keyword_li)

df = pd.DataFrame.from_dict(keyword_count, orient='index').reset_index()
df.columns = ['key_word', 'count']
df = df.sort_values(by='count', ascending=False)
df = df.query("count>1")
df = df.reset_index(drop=True)

base_html = """
<!doctype html>
<html>
    <head>
        <meta http-equiv="Content-type" content="text/html; charset=utf-8">
        <script type="text/javascript" src="https://ajax.googleapis.com/ajax/libs/jquery/2.2.2/jquery.min.js"></script>
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.16/css/jquery.dataTables.css">
        <script type="text/javascript" src="https://cdn.datatables.net/1.10.16/js/jquery.dataTables.js"></script>
    </head>
    <body>%s
        <script type="text/javascript">$(document).ready(function(){$('table').DataTable({
            "pageLength": 10
        });});
        </script>
    </body>
</html>
"""

def df_html(df):
    """HTML table with pagination and other goodies"""
    df_html = df.to_html()
    return base_html % df_html
  
with open('./interactive_df.html','w', encoding='utf8') as f:
    f.write(df_html(df))

In [9]:
import plotly as py
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import time as dt_tm
from datetime import date as dt_date
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import os
import tempfile
os.environ['MPLCONFIGDIR'] = tempfile.mkdtemp()
import re


with open('./action_count.html', 'r') as f:
    action_count = f.read()
    
with open('./interactive_df.html', 'r', encoding='utf8') as f:
    interactive_df = f.read()
    
html_string = '''
<!doctype html>
<html>
    <head>
        <meta charset="UTF-8">
        <title>Dashboard😱</title>
        <style>body{ margin:0 100; background:whitesmoke; }</style>
    </head>
    
    <body>
        <h1>Web Analytics</h1>
        <!-- *** Section 1 *** --->
        <h2>Action Count</h2>
        ''' + action_count + '''
        <p>graph description...</p>
         
        <!-- *** Section 2 *** --->
        <h2>Interactive Table</h2>
        ''' + interactive_df + '''
    
    </body>
</html>'''

with open('./report.html', 'w', encoding = 'utf8') as f:
    f.write(html_string)